In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy

import warnings; warnings.simplefilter('ignore')

In [2]:
#단순추천시스템 (평점순, 장르별)
#TMDB ratings 사용

In [3]:
md = pd.read_csv('/Users/zogak/movie_archive/movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#dictionary안에 담겨있는 Genre정보를 List형태로 
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

print(md['genres'].head())

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object


In [5]:
#m : 영화가 차트에 오르기 위해 얻어야 하는 최소 투표 수
# -> 전체 영화의 95% 이상의 투표를 얻어야 한다고 가정

In [6]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [7]:
#상위 5%에 해당하는 값
m = vote_counts.quantile(0.95)
m

434.0

In [8]:
#release date에서 연도만 추출
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x : str(x).split('-')[0] if x != np.nan else np.nan)
print(md['year'].head())

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object


In [9]:
#상위 5%까지 데이터 추출
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [10]:
#차트에 오르려면 434개 이상의 평가를 받아야 함
#영화 평균 평점은 10점 만점에 5.244
#2274편의 영화가 차트에 올랐음

In [11]:
#v : 영화에 대한 평가 수
#m : 차트에 오르기 위해 가져야하는 최소 평가 수
#R : 영화 평점
#C : 전체 영화에 대한 평균 평점

In [12]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [13]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [14]:
#weighted rating 상위 250개 영화
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [15]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


In [16]:
s = md.apply(lambda x : pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

gen_md = md.drop('genres', axis=1).join(s)

In [17]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[ (df['vote_count'] >=m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x :(x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count'])*C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [18]:
build_chart('Romance').head(10)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


In [19]:
#콘텐츠 기반 추천

In [20]:
links_small = pd.read_csv('/Users/zogak/movie_archive/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [21]:
md = md.drop([19730,29503,35587])

In [22]:
md['id'] = md['id'].astype('int')

In [23]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [24]:
#영화 내용 기반 추천 시스템

In [25]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

smd['description'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: description, dtype: object

In [26]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [27]:
print(tfidf_matrix[10])

  (0, 237880)	0.14118434571637967
  (0, 264587)	0.15239448296082492
  (0, 149397)	0.1346268358004416
  (0, 184592)	0.13757548758984955
  (0, 192607)	0.15239448296082492
  (0, 12515)	0.15239448296082492
  (0, 59410)	0.15239448296082492
  (0, 204261)	0.15239448296082492
  (0, 260369)	0.15239448296082492
  (0, 221301)	0.15239448296082492
  (0, 51585)	0.15239448296082492
  (0, 15732)	0.15239448296082492
  (0, 213978)	0.15239448296082492
  (0, 142130)	0.15239448296082492
  (0, 256956)	0.14583697304488685
  (0, 255009)	0.15239448296082492
  (0, 73255)	0.15239448296082492
  (0, 232769)	0.15239448296082492
  (0, 51809)	0.15239448296082492
  (0, 255952)	0.15239448296082492
  (0, 154810)	0.14583697304488685
  (0, 184597)	0.14583697304488685
  (0, 264425)	0.13213378623513553
  (0, 213982)	0.15239448296082492
  (0, 10509)	0.15239448296082492
  :	:
  (0, 259727)	0.15239448296082492
  (0, 237762)	0.08073572734141646
  (0, 192601)	0.13213378623513553
  (0, 12513)	0.13757548758984955
  (0, 59409)	0.14

In [28]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [30]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

print(titles.head(), indices.head())

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [31]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [32]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [33]:
get_recommendations('Inception').head(10)

5239                              Cypher
141                                Crumb
6398                         Renaissance
653                            Lone Star
1703                               House
4739                    The Pink Panther
319                                 Cobb
2828    What Ever Happened to Baby Jane?
8867                     Pitch Perfect 2
979          Once Upon a Time in America
Name: title, dtype: object

In [34]:
#메타 데이터 기반 추천시스템

In [35]:
credits = pd.read_csv('/Users/zogak/movie_archive/credits.csv')
keywords = pd.read_csv('/Users/zogak/movie_archive/keywords.csv')

In [36]:
credits['crew'][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [37]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [38]:
md.shape

(45463, 25)

In [39]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [40]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [41]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x : len(x))
smd['crew_size'] = smd['crew'].apply(lambda x : len(x))

In [42]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
        return np.nan

In [43]:
smd['director'] = smd['crew'].apply(get_director)

In [44]:
smd['cast'] = smd['cast'].apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x : x[:3] if len(x) >= 3 else x)

In [45]:
smd['keywords'] = smd['keywords'].apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

In [46]:
smd['cast'] = smd['cast'].apply(lambda x : [str.lower(i.replace(" ","")) for i in x])

In [47]:
smd['director'] = smd['director'].astype('str').apply(lambda x : str.lower(x.replace(" ","")))
smd['director'] = smd['director'].apply(lambda x :[x, x, x])

In [48]:
s = smd.apply(lambda x : pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [49]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [50]:
s = s[s>1]

In [51]:
stemmer = SnowballStemmer('english')
print("dogs의 어근 : ",stemmer.stem('dogs'))
print("dogs의 어근: ", stemmer.stem('dog'))

dogs의 어근 :  dog
dogs의 어근:  dog


In [52]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [53]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x :[stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x : [str.lower(i.replace(" ","")) for i in x])

In [54]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x : ' '.join(x))

In [56]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [57]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [58]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [59]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
5098                  The Enforcer
6218                 Batman Begins
5943                      Thursday
217               The Glass Shield
2272                   In Too Deep
6349           Menolippu Mombasaan
8026            Bullet to the Head
7659    Batman: Under the Red Hood
5073                 The Statement
Name: title, dtype: object

In [60]:
get_recommendations('Mean Girls').head(10)

8073           Dylan Moran: What It Is
5214              Swimming to Cambodia
7430       Werner Herzog Eats His Shoe
7470                           The Fly
8213                       Lola Versus
418                      House Party 3
2671                     Class Reunion
2781    National Lampoon's Last Resort
3092                        Easy Money
3428                      Nine to Five
Name: title, dtype: object